# **Welcome to MAPSHB**

This is a Jupyter notebook for running the MAPSHB model using the AmberTools23 engine and the R and Fortran languages. This notebook implements the model detailed in the paper "***Effective prediction of short hydrogen bonds in proteins via machine learning method***" ([link here](https://www.nature.com/articles/s41598-021-04306-4)).

---
**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/shengminz/MAPSHB/issues

**Emails**
- **Shengmin Zhou** ([shengminzhou93@gmail.com](mailto:shengminzhou93@gmail.com))
- **Yuanhao Liu** ([yl1398@scarletmail.rutgers.edu](mailto:yl1398@scarletmail.rutgers.edu))
- **Sijian Wang** ([sijian.wang@stat.rutgers.edu](mailto:sijian.wang@stat.rutgers.edu))
- **Lu Wang** ([lwang@chem.rutgers.edu](mailto:lwang@chem.rutgers.edu))

In [ ]:
#@title **Install Conda Colab**
#@markdown It will automatically restart the kernel (session) after finishing the installation. After completion, you may see the following message in the bottom right of your browser:

#@markdown Your session crashed. Automatically restarting.

#@markdown Your session restarted after a crash. Diagnosing...

#@markdown Your session crashed for an unknown reason. View runtime logs

#@markdown It doesn't matter and don't worry. Just continue to run the next cell.

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [ ]:
#@title **Install dependencies**
#@markdown Install AmberTools23, R and Fortran. Download MAPSHB source codes from Github.

#@markdown It will take a few minutes (~3 mins), please wait.

# install dependencies
%%capture
!mamba install -c conda-forge ambertools -y
!git clone https://github.com/shengminz/MAPSHB.git
!mv MAPSHB/* .

#load dependencies
import sys
import os
import urllib.request
import numpy as np

!apt-get update -qq
!apt-get install -y r-base
!pip install -q rpy2
!apt-get -qq install gfortran
#!conda install -c r r-base -y
#!conda install -c conda-forge gfortran_linux-64 -y

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Install the 'gbm' package in R
utils = importr('utils')
utils.install_packages('gbm')

In [ ]:
#@title ### **Import input file**
#@markdown Click in the "Run" buttom and click "Choose Files" to upload your PDB file from local.

#@markdown **Before upload your own PDB file**, make sure you have already read the following instructions about input file:

#@markdown The uploaded protein structure must be in the PDB format. The protein structure must contain all atoms, including the hydrogen atoms in amino acid residues and small molecule ligands. The hydrogen atoms are used for the determination of hydrogen bonds. You can add them to the amino acid residues using a software of your choice prior to this analysis. For example, in ChimeraX, you can type "addh" to add hydrogens, and in AmberTools, you can use "reduce" to add hydrogens. Then you can save the structure as a PDB file.

from google.colab import files

# Upload file from your local machine
uploaded = files.upload()

# Process the uploaded file
for filename in uploaded.keys():
    # Rename the file to "input.pdb"
    new_filename = "input.pdb"
    with open(new_filename, "wb") as f:
        f.write(uploaded[filename])
    print(f"Uploaded file: {filename} (copied as {new_filename})")


Saving 1c57.pdb to 1c57.pdb
Uploaded file: 1c57.pdb (copied as input.pdb)


In [ ]:
#@title ### **Run Prediciton**
#@markdown Click in the "Run" buttom to run prediction. (~1 min)

#@markdown Make sure there's no error file "error.log" generated by this step, otherwise check your input file.

%%capture
!grep "ATOM  " input.pdb | grep "      " > aa.pdb
!sed -i '/REMARK/d' aa.pdb
!sed -i '/REVDAT/d' aa.pdb
!sed -i '/CAVEAT/d' aa.pdb
!pdb4amber -i aa.pdb -o amber.pdb -d -p --noter

f = open("tleap.in", "w")
f.write("source leaprc.protein.ff14SB" + "\n")
f.write("x=loadpdb amber.pdb" + "\n")
f.write("saveamberparm x amber.parm amber.rst" + "\n")
f.write("quit" + "\n")
f.close()

!tleap -f tleap.in

f = open("cpptraj.in", "w")
f.write("parm amber.parm" + "\n")
f.write("trajin amber.rst" + "\n")
f.write("hbond dist +3.2 avgout hbond" + "\n")
f.write("secstruct assignout secstruct" + "\n")
f.write("run" + "\n")
f.write("quit" + "\n")
f.close()

!cpptraj -i cpptraj.in
!awk -v OFS='\t' 'NR==1{for (i=1;i<=NF;i++)if ($i=="DonorH"){n=i-1;m=NF-(i==NF)}} {for(i=1;i<=NF;i+=1+(i==n))printf "%s%s",$i,i==m?ORS:OFS}' hbond > temp
!awk -v OFS='\t' 'NR==1{for (i=1;i<=NF;i++)if ($i=="Frames"){n=i-1;m=NF-(i==NF)}} {for(i=1;i<=NF;i+=1+(i==n))printf "%s%s",$i,i==m?ORS:OFS}' temp > hbond
!awk -v OFS='\t' 'NR==1{for (i=1;i<=NF;i++)if ($i=="Frac"){n=i-1;m=NF-(i==NF)}} {for(i=1;i<=NF;i+=1+(i==n))printf "%s%s",$i,i==m?ORS:OFS}' hbond > temp
!mv temp hbond
!sed -i '/AvgDist/d' hbond
!find -name hbond | xargs perl -pi -e 's|_|   |g'
!find -name hbond | xargs perl -pi -e 's|@|   |g'
!column -t < hbond > temp
!mv temp hbond
!cut -c 18-26 < aa.pdb > temp
!perl -ne 'print unless $dup{$_}++;' temp > seq_origin
!find -name seq_origin | xargs perl -pi -e 's|HIE|HIS|g'
!find -name seq_origin | xargs perl -pi -e 's|HID|HIS|g'
!find -name seq_origin | xargs perl -pi -e 's|HIP|HIS|g'
!find -name seq_origin | xargs perl -pi -e 's|CYX|CYS|g'
!sed -i '/TER/d' amber.pdb
!sed -i '/END/d' amber.pdb
!cut -c 18-26 < amber.pdb > temp
!perl -ne 'print unless $dup{$_}++;' temp > seq_amber
!find -name seq_amber | xargs perl -pi -e 's|HIE|HIS|g'
!find -name seq_amber | xargs perl -pi -e 's|HID|HIS|g'
!find -name seq_amber | xargs perl -pi -e 's|HIP|HIS|g'
!find -name seq_amber | xargs perl -pi -e 's|CYX|CYS|g'
!sed -i '/^\s*$/d' seq_amber

seq_origin_file = "seq_origin"
seq_amber_file = "seq_amber"
with open("seq_origin") as f_origin:
  num_lines_origin = [line for line in f_origin.readlines()]
with open("seq_amber") as f_amber:
  num_lines_amber = [line for line in f_amber.readlines()]
with open("hbond") as f_hbond:
  num_lines_hbond = [line for line in f_hbond.readlines()]
if len(num_lines_origin) != len(num_lines_amber):
#    print("Error 1: Non-Amber-compatible residues marked as ATOM. Please change it into HETATM.")
    with open("error.log", "w") as f.error:
      f.error.write("Error 1: Non-Amber-compatible residues marked as ATOM. Please change it into HETATM.")

!cut -c 10-19,21-30,32-41,43-52,54-63 < secstruct > temp_ss
!cp prepare.f90 temp.f90

with open("temp.f90", "r") as file:
    content = file.read()

# Replace 'seq_number' and 'hb_number' with 'a' and 'b' respectively in the content
content = content.replace("seq_number", str(len(num_lines_origin)))
content = content.replace("hb_number", str(len(num_lines_hbond)))

# Write the modified content back to "temp.f90" file
with open("temp.f90", "w") as file:
    file.write(content)

!gfortran temp.f90
!./a.out
!cut -c 1-37,42-65,70-108 < output.log > temp

# Now you can run your R script
!wget https://zenodo.org/record/8248034/files/Copyright.log
!Rscript boosting_online.r
#robjects.r.source('boosting_online.r')

In [ ]:
#@title ### **Display and download result**
#@markdown Click in the "Run" buttom to display and download the prediction.

#@markdown An output file "predicit_results.csv" will be downloaded to your computer.

#@markdown The output file is in the comma-separated value (.csv) format. Each line contains the information of a hydrogen bond.

#@markdown The first three columns include the donor/acceptor residue names, donor/acceptor residue numbers, donor/acceptor atom names (after the @ sign) and the hydrogen bond distance from your input PDB structure. The hydrogen bond distances (R) are for your reference, and they are not used in our predictions.

#@markdown The next two columns report the probability of this hydrogen bond to form a SHB and a recommended hydrogen bond class from the MAPSHB model. The recommended hydrogen bond class is based on a classification threshold of 0.870. The models identify the hydrogen bond as a SHB if its probability is ≥ the classificatioin threshold, or a NHB if the probability is below the threshold. You can choose a different threshold to define the hydrogen bond class, as discussed on https://wanggroup.rutgers.edu/mapshb-model/the-mapshb-model/17-research/mapshb-model/41-probability-thresholds.
!cat predict_results.csv
files.download(f"predict_results.csv")

"donor residue/atom","acceptor residue/atom","R from structure (A)","Probability of forming a SHB","Recommended hydrogen bond class (Probability threshold = 0.892)"
"SER_169@OG","ASP_145@OD2",2.62,"0.998","SHB"
"SER_110@OG","SER_110@O",3.18,"0.915","SHB"
"SER_160@OG","SER_164@O",3.11,"0.897","SHB"
"ASN_104@ND2","GLU_102@OE2",2.57,"0.894","SHB"
"THR_196@OG1","THR_196@O",3.09,"0.789","NHB"
"LYS_138@NZ","ASP_136@OD2",3.12,"0.758","NHB"
"HIS_24@ND1","TYR_22@O",2.8,"0.369","NHB"
"HIS_127@ND1","THR_112@OG1",2.83,"0.295","NHB"
"HIS_51@ND1","THR_194@OG1",3.15,"0.275","NHB"
"ARG_60@NE","ASP_78@OD1",2.98,"0.112","NHB"
"ARG_228@NH1","ASP_16@OD2",2.78,"0.042","NHB"
"ARG_172@NE","GLN_143@OE1",2.98,"0.018","NHB"
"ARG_172@NH1","GLY_231@O",2.96,"0.017","NHB"
"GLN_137@NE2","ASP_139@OD1",2.89,"0.014","NHB"
"ASN_55@ND2","ASP_58@OD2",3.06,"0.009","NHB"
"ARG_90@NH1","ILE_217@O",2.76,"0.006","NHB"
"ARG_90@NH2","TYR_176@O",2.83,"0.005","NHB"
"ARG_172@NH2","SER_219@OG",2.91,"0.005","NHB"
"ARG_90@NE","TYR_176@

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>